## Insert PatientDatasets into Database
- [ ] exp.json => Exps
- [ ] info.json => Patients
- [ ] normal.csv => Inspections

In [39]:
import pandas as pd
import json
import glob
import os
from IPython.display import display
from django.shortcuts import render, get_object_or_404, get_list_or_404

In [7]:
def InsertPatient(info_json, patient_id):
    info_json["patient_text"] = "patient " + str(info_json["patient_id"])
    del info_json["patient_id"]
    result, created = Patient.objects.update_or_create(
        id=patient_id,
        defaults=info_json,
    )

def InsertExp(exp_json, patient_id):
    """
    ExpをDatabaseにInsert
    arg:
    exp_json ( = '{"ALB t_0 < 3.3": 0.1,"BUN": 0.05}')
    """
    
    for k, v in exp_json.items():
        Exp.objects.update_or_create(
            patient=Patient(patient_id),
            cond_exp=k,
            defaults={"contribution":v},
        )

        
        
def InsertInspection(patient_df, patient_id):
    """
    InspectionをDatabaseにInsert
    arg:
    patient_df (require : index = Time, cloums = InspectionItems)
    """
    
    for item in patient_df:
        for date, value in patient_df[item].items():
            Inspection.objects.update_or_create(
                patient=Patient(patient_id),
                inspection_date = date,
                inspection_item = item,
                defaults={"inspection_value" : value},
            )

def insert_dataset():
    head_dir = "./PatientsDataset"
    query = "patient*"
    normal_file = "normal.csv"
    info_file = "info.json"
    exp_html_file = "exp.html"
    exp_json_file = "exp.json"
    patient_dir_list = glob.glob(os.path.join(head_dir,query))
    for patient_dir in patient_dir_list:
        patient_id = int(os.path.basename(patient_dir)[7:])
        normal_path = os.path.join(patient_dir, normal_file)
        info_path = os.path.join(patient_dir, info_file)
        exp_html_path = os.path.join(patient_dir, exp_html_file)
        exp_json_path = os.path.join(patient_dir, exp_json_file)
        patient_df = pd.read_csv(normal_path, index_col=0)
        if patient_id == 10000:
            pass
        with open(info_path, "r") as f:
            info_json = json.load(f)
        with open(exp_json_path, "r") as f:
            exp_json = json.load(f)
        InsertPatient(info_json, patient_id)
        InsertExp(exp_json, patient_id)
        InsertInspection(patient_df, patient_id)
        
        

In [8]:
insert_dataset()

## Test
色々実験する

In [9]:
patient_num = Patient.objects.count() // 3

In [11]:
div = Patient.objects.count() // 3

In [20]:
Patient.objects.values_list("")

<QuerySet [(10000,), (10001,), (10002,), (10003,), (10004,), (10005,), (10006,), (10007,), (10008,), (10009,)]>

In [36]:
test_id = "A"
order_value = 1

In [49]:
div = Patient.objects.count() // 3
# order_value = request.user.id % 3
base_dict = {"A":0, "B":1, "C":2}
start = ((base_dict[test_id] + order_value) % 3) * div
end = start + div
# patient_list = Patient.objects.values_list()[start: end]
patient_list = get_list_or_404(Patient)[start:end]

In [51]:
patient_list

[<Patient: patient 10003>, <Patient: patient 10004>, <Patient: patient 10005>]

In [54]:
test_list = get_list_or_404(Inspection, patient=10001)

In [57]:
test_list

[<Inspection: name:patient 10001 | date:2019-11-14 | item:ALB>,
 <Inspection: name:patient 10001 | date:2019-11-15 | item:ALB>,
 <Inspection: name:patient 10001 | date:2019-11-16 | item:ALB>,
 <Inspection: name:patient 10001 | date:2019-11-17 | item:ALB>,
 <Inspection: name:patient 10001 | date:2019-11-18 | item:ALB>,
 <Inspection: name:patient 10001 | date:2019-11-19 | item:ALB>,
 <Inspection: name:patient 10001 | date:2019-11-20 | item:ALB>,
 <Inspection: name:patient 10001 | date:2019-11-21 | item:ALB>,
 <Inspection: name:patient 10001 | date:2019-11-22 | item:ALB>,
 <Inspection: name:patient 10001 | date:2019-11-23 | item:ALB>,
 <Inspection: name:patient 10001 | date:2019-11-24 | item:ALB>,
 <Inspection: name:patient 10001 | date:2019-11-25 | item:ALB>,
 <Inspection: name:patient 10001 | date:2019-11-26 | item:ALB>,
 <Inspection: name:patient 10001 | date:2019-11-14 | item:BUN>,
 <Inspection: name:patient 10001 | date:2019-11-15 | item:BUN>,
 <Inspection: name:patient 10001 | date:

In [39]:
test = Inspection.objects.filter(patient=10001, inspection_item="ALB").values_list("inspection_date", flat=True)

In [40]:
for w in test:
    print(w)

2019-11-14
2019-11-15
2019-11-16
2019-11-17
2019-11-18
2019-11-19
2019-11-20
2019-11-21
2019-11-22
2019-11-23
2019-11-24
2019-11-25
2019-11-26
2020-07-13


In [68]:
inspections = Inspection.objects.filter(patient=10001).order_by("inspection_item", "inspection_date")

In [78]:
item_list = ["ALB","BUN","CRE","CRP","Ca","Cl","IP","K","Na","T-CHO","PCR", "TP"]

In [82]:
patient_id = 10001

In [83]:
inspection_array = [Inspection.objects.filter(patient=patient_id, inspection_item=item).order_by("inspection_item").values_list("inspection_value", flat=True) for item in item_list]

In [93]:
for w in inspection_array[-2]:
    print(w)

532.7264744400383
868.6708342917128
643.2449991513897
741.2646839361197
1071.7092420905212
1018.8984955336026
1045.1286128059241
231.01155425407208
1053.588974795359
876.1678292660613
777.847931829908
1109.0951806553371
770.2579246010814
None


In [62]:
for date in inspections.order_by("inspection_date").values_list("inspection_date", flat=True).distinct():
    print(date)

2019-11-14
2019-11-15
2019-11-16
2019-11-17
2019-11-18
2019-11-19
2019-11-20
2019-11-21
2019-11-22
2019-11-23
2019-11-24
2019-11-25
2019-11-26
2020-07-13


In [66]:
oneitem_inspection_list = inspections.get(inspection_item="ALB", inspection_date="2019-11-14")

In [67]:
oneitem_inspection_list.inspection_value

4.1361581563891665

In [15]:
oneitem_inspection_list.order_by("inspection_date")

<QuerySet [<Inspection: name:patient 10001 | date:2019-11-14 | item:ALB>, <Inspection: name:patient 10001 | date:2019-11-15 | item:ALB>, <Inspection: name:patient 10001 | date:2019-11-16 | item:ALB>, <Inspection: name:patient 10001 | date:2019-11-17 | item:ALB>, <Inspection: name:patient 10001 | date:2019-11-18 | item:ALB>, <Inspection: name:patient 10001 | date:2019-11-19 | item:ALB>, <Inspection: name:patient 10001 | date:2019-11-20 | item:ALB>, <Inspection: name:patient 10001 | date:2019-11-21 | item:ALB>, <Inspection: name:patient 10001 | date:2019-11-22 | item:ALB>, <Inspection: name:patient 10001 | date:2019-11-23 | item:ALB>, <Inspection: name:patient 10001 | date:2019-11-24 | item:ALB>, <Inspection: name:patient 10001 | date:2019-11-25 | item:ALB>, <Inspection: name:patient 10001 | date:2019-11-26 | item:ALB>, <Inspection: name:patient 10001 | date:2020-07-13 | item:ALB>]>

In [18]:
for w in oneitem_inspection_list.order_by("inspection_date"):
    print(w)

name:patient 10001 | date:2019-11-14 | item:ALB
name:patient 10001 | date:2019-11-15 | item:ALB
name:patient 10001 | date:2019-11-16 | item:ALB
name:patient 10001 | date:2019-11-17 | item:ALB
name:patient 10001 | date:2019-11-18 | item:ALB
name:patient 10001 | date:2019-11-19 | item:ALB
name:patient 10001 | date:2019-11-20 | item:ALB
name:patient 10001 | date:2019-11-21 | item:ALB
name:patient 10001 | date:2019-11-22 | item:ALB
name:patient 10001 | date:2019-11-23 | item:ALB
name:patient 10001 | date:2019-11-24 | item:ALB
name:patient 10001 | date:2019-11-25 | item:ALB
name:patient 10001 | date:2019-11-26 | item:ALB
name:patient 10001 | date:2020-07-13 | item:ALB


In [7]:
insert_objs

<QuerySet []>

In [26]:
os.path.exists("./PatientsDataset")

True

In [27]:
!ls PatientsDataset/

patient10000 patient10002 patient10004 patient10006 patient10008
patient10001 patient10003 patient10005 patient10007 patient10009


In [12]:
test = './PatientsDataset/patient1'

In [15]:
os.path.basename(test)[7:]

'1'

In [14]:
import re